<a href="https://colab.research.google.com/github/DABallentine/knowledge_discovery_charlotte/blob/origin%2FNaomiT/Jupiter%20Notebooks/EDA_and_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Charlotte 311 Service Request Analysis

<h1> <b> Exploratory Data Analysis and Preprocessing </b> </h1>

## Data Import and Survey

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [2]:
#function to read data from URL
def read_data_from_URL(url):
    df=pd.read_csv(url)
    return df

In [3]:
serv_req_url='https://bitbucket.org/nthammad-uncc/test_repo/raw/7508681846896228bb87fd1989ec50ca2d592bc5/data/Service_Requests_311.csv'
#url2='https://drive.google.com/file/d/1NVN7kq_qpIwkW3z-47Z27VTgFrKvuhCs/view?usp=sharing'
#drive_url='https://drive.google.com/uc?id='+url2.split('/')[-2]
requests = read_data_from_URL(serv_req_url)
requests.head()

,X,Y,OBJECTID,DEPARTMENT,DIVISION,REQUEST_NO,FISCAL_YEAR,FISCAL_MONTH,REQUEST_TYPE,RECEIVED_DATE,...,ZIP_CODE,FULL_ADDRESS,X_COORD,Y_COORD,LATITUDE,LONGITUDE,COUNCIL_DISTRICT,GlobalID,PID,Shape
0,1.458251e+06,508075.000119,1,Solid Waste Services,Sanitation,6402056,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:25:00+00,...,28226.0,"3601 KNAPDALE LN, CHARLOTTE, NC 28226",1458251.0,508075.0,35.132444,-80.812089,NaN,{35439C7B-D8E4-4296-9119-8673B1D5A7C8},20904304,NaN
1,1.468687e+06,587225.999953,2,Housing and NBHD Serv,311 Partner Liaison,6402064,2017,05-Nov,UTILITY VERIFICATION LTR,2016/11/07 14:28:00+00,...,28269.0,"4124 SILVERMERE WY, CHARLOTTE, NC 28269",1468687.0,587226.0,35.350408,-80.781941,NaN,{3B4A2DC0-8886-4E8E-95E6-4F426C4E08E2},02731251,NaN
2,1.471481e+06,562512.999839,3,Solid Waste Services,Sanitation,6402082,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:34:00+00,...,28213.0,"523 GRAHAM MEADOW DR, CHARLOTTE, NC 28213",1471481.0,562513.0,35.282654,-80.771094,NaN,{C3E4CF76-7630-4356-8216-428445A0903F},08917486,NaN
3,1.438992e+06,568604.000124,4,Solid Waste Services,Sanitation,6402103,2017,05-Nov,MISSED RECYCLING,2016/11/07 14:41:00+00,...,28216.0,"1801 CRANDON DR, CHARLOTTE, NC 28216",1438992.0,568604.0,35.297745,-80.880327,NaN,{FC3B4CD8-1131-42B4-A37E-8CA448855B24},03925515,NaN
4,NaN,NaN,5,Transit,Transit/Operations,6402111,2017,05-Nov,CITIZEN FEEDBACK,2016/11/07 14:44:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{E0166346-2381-40E1-80A0-2ECA4FB6CB27},NaN,NaN


In [4]:
print("Records:", requests.shape[0], "\nFeatures:", requests.shape[1])
print(requests.info())

Records: 1631510 
Features: 30
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631510 entries, 0 to 1631509
Data columns (total 30 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   X                           1559084 non-null  float64
 1   Y                           1559084 non-null  float64
 2   OBJECTID                    1631510 non-null  int64  
 3   DEPARTMENT                  1631510 non-null  object 
 4   DIVISION                    1623095 non-null  object 
 5   REQUEST_NO                  1631510 non-null  int64  
 6   FISCAL_YEAR                 1631510 non-null  int64  
 7   FISCAL_MONTH                1631510 non-null  object 
 8   REQUEST_TYPE                1631510 non-null  object 
 9   RECEIVED_DATE               1631510 non-null  object 
 10  INTERNAL_FIELD_OBSERVATION  1631510 non-null  object 
 11  TITLE                       1631510 non-null  object 
 12  CMPD_DIVISION            

## Initial Variable Selection


<b> The following variables will be dropped: </b>
<ol>
<li> X - redundant </li>
<li> Y - redundant </li>
<li> CMPD_DIVISION - too many missing values </li>
<li> STREET_DIRECTION - redundant </li>
<li> STREET_NAME - redundant </li>
<li> STREET_TYPE - redundant </li>
<li> CITY - redundant </li>
<li> STATE - redundant </li>
<li> COUNCIL_DISTRICT - too many missing values </li>
<li> GLOBAL_ID - no predictive value </li>
<li> PID - no predictive value </li>
<li> SHAPE - all missing </li>
</ol>

In [5]:
requests.drop(requests.iloc[:, [0,1,4,12,15,16,17,18,19,26,27,28,29]], axis=1, inplace=True)
requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631510 entries, 0 to 1631509
Data columns (total 17 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   OBJECTID                    1631510 non-null  int64  
 1   DEPARTMENT                  1631510 non-null  object 
 2   REQUEST_NO                  1631510 non-null  int64  
 3   FISCAL_YEAR                 1631510 non-null  int64  
 4   FISCAL_MONTH                1631510 non-null  object 
 5   REQUEST_TYPE                1631510 non-null  object 
 6   RECEIVED_DATE               1631510 non-null  object 
 7   INTERNAL_FIELD_OBSERVATION  1631510 non-null  object 
 8   TITLE                       1631510 non-null  object 
 9   NEIGHBORHOOD_PROFILE_AREA   1559062 non-null  float64
 10  BLOCK_NO                    1587157 non-null  float64
 11  ZIP_CODE                    1586427 non-null  float64
 12  FULL_ADDRESS                1624342 non-null  object 
 1

## Survey Request Types and Departments

In [6]:
print("Total unique request categories:", requests['REQUEST_TYPE'].unique().shape[0], "\n")
print("Top 30 categories by volume:\n")
print(requests['REQUEST_TYPE'].value_counts().head(30), "\n")

print("Top 30 departments by volume:\n")
print(requests['DEPARTMENT'].value_counts().head(30))

Total unique request categories: 165 

Top 30 categories by volume:

NON_RECYCLABLE ITEMS         831737
RECYCLABLE ITEMS             199452
HNS HEALTH AND SANITATION     85031
CART REPAIR-GARBAGE           36541
MISSED RECYCLING              27471
SW DOCUMENT GARBAGE           26729
311 DOCUMENT                  25395
CDOT STREETLIGHT REPAIR       19216
SW DOCUMENT RECYCLING         17932
DEAD ANIMAL COLLECTION        16727
CART ISSUE/MISSING CART       15121
TIRES                         13577
SW DOCUMENT YARD WASTE        13186
CART DELIVERY RECYCLING       11835
COMPLAINT                     11788
ZONING/COMPLAINT/ INSPECT     11767
CART(S) DEL- NEW BUILD        11324
CART ONLY DELIVER-GARB        11235
CART REPAIR-RECYCLE            9933
ZON GENERAL INFO  INQUIRY      9805
CDOT POTHOLE REPAIR            9416
HNS HOUSING                    8405
CART PURCHASE 2ND-GARBAGE      8077
MISSED GARBAGE                 7330
CDOT STREET SIGN REP/REPL      7260
PARKING ON LAWN WORKDAYS       

In [7]:
print("Bottom 20 categories by volume:\n")
print(requests['REQUEST_TYPE'].value_counts().tail(30).sort_values(ascending=True))

Bottom 20 categories by volume:

SWS MLTFMLY FIELD OBSERVE     1
BUS STOP-REMOVE TRASH CAN     1
SW EXCESSIVE STORM-SW USE     1
UT/MANDATORY                  1
UT/LANDSCAPE                  1
DISABILITY ACT                1
COMPLEX PROBLEM               1
SW FIELD OBSERVED TO CI       1
ON CALL - SWS SPEC SERV       1
SW REMINDER BULKY-SW USE      2
SW REMINDER GARB-SW USE       4
NEW SVC BUSINESS GARBAGE      5
SW ESCALATION REQUEST         6
CART REPAIR-BUSN GARB         6
BUS STOP-TRASH CAN MISC       6
CART DELIVERY BUSN GARB       7
OPINION                       7
BUS FACILITIES MAINT          8
BULKY ITEM SCHEDULE           8
LAND DEVELOPMENT INQUIRY      8
SMOKING VEHICLE REPORT        9
UT RATE CHANGE                9
CART ONLY DELIVER-RECY       10
BUS STOP-FULL TRASH  CAN     10
MUD IN THE STREET            10
PARK & REC REFUND/CANCEL     11
SWS CART RETIRE REVIEW       12
DISABLED INSPECTION (SWS)    12
ZON APPEALS/CITATIONS INQ    14
CWP REQUEST-HIGH PRIORITY    16
Name: R

### Display all 311 request types alphabetically

In [8]:
# Set Display Options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Subset requests by unique values, sorted
A_CART = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[0:25].reset_index()
CDOT_COM = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[25:50].reset_index()
CON_INV = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[50:75].reset_index()
INV_PAR = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[75:100].reset_index()
PARK_SWDOC = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[100:125].reset_index()
SWESC_ZON = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[125:150].reset_index()
ZON_ZON = pd.Series(requests['REQUEST_TYPE'].unique()).sort_values()[150:165].reset_index()

# Combine those 7 Series of 25 values each into 1 data frame for convenient side-by-side display 
categories = pd.concat([A_CART, CDOT_COM, CON_INV, INV_PAR, PARK_SWDOC, SWESC_ZON, ZON_ZON], axis=1)
categories.drop(categories.iloc[:,[0,2,4,6,8,10,12]], axis=1, inplace=True) # drop the index columns
categories.columns = ['A-CART', 'CDOT-COM', 'CON-INV', 'INV-PAR', 'PARK-SWDOC', 'SWESC-ZON', 'ZON-ZON']
categories.reset_index(drop=True)
categories = categories.style.set_properties(**{'text-align': 'left'}) # left justify for easy comparison of topics to determine possible bins / combinations
categories

,A-CART,CDOT-COM,CON-INV,INV-PAR,PARK-SWDOC,SWESC-ZON,ZON-ZON
0,3-1-1 ESCALATION REQUEST,CDOT BASKETBALL,CONTENT IMPROVEMENT PROG,INVESTIGATE/CONTACT GARB,PARKING ON LAWN OFFHOURS,SW ESCALATION REQUEST,ZON GENERAL INFO INQUIRY
1,311 DOCUMENT,CDOT CLEAN CATCH BASIN,COVID 19-PRICE GOUGING,INVESTIGATE/CONTACT RECY,PARKING ON LAWN WORKDAYS,SW EXCESSIVE STORM-SW USE,ZON HOLDS RELEASE REQUEST
2,ADOPT-A PICKUP SWS,CDOT FLOODING,COVID 19-PUBLIC GATHERING,INVESTIGATE/CONTACT YD WT,PLANNING INQUIRY,SW FIELD OBSERVED TO CI,ZON RESIDENTIAL PERMITS
3,ANIMAL FOUND REPORT,CDOT GRATE DAMAGE/REPLACE,COVID 19-STAY HOME VIOL,LAND DEVELOPMENT INQUIRY,POLLUTION,SW ONLY-DOOR HANGER LEFT,ZON SETBACKS
4,BLOCKAGE,CDOT GUARD RAIL,CURRENT DISABLED SERVICE,LANDSCAPE LIMBS,RECYCLABLE ITEMS,SW REMINDER BULKY-SW USE,ZON SIGNS INQUIRY
5,BOARDED UP STRUCTURE,CDOT MISCELLANEOUS,CWP REQUEST,LANDSCAPE TREES,REQUEST DISABILITY FORM,SW REMINDER GARB-SW USE,ZON STORAGE BLDG/ACC UNIT
6,BULKY ITEM SCHEDULE,CDOT NEW CROSSWALK,CWP REQUEST-HIGH PRIORITY,LITTER/DEBRIS IN STREET,REQUEST FOR SPEAKER,SW REMINDER RECY-SW USE,ZON SUBDIVIDE PROP/DUPLX
7,BUS FACILITIES MAINT,CDOT NEW SIDEWALK,DAMAGE/INVESTIGATE BULKY,MISSED BULKY ITEM,SIDEWALK OBSTR CONTAINERS,SW REMINDER YARD-SW USE,ZON SUPERVISORY COMPLAINT
8,BUS STOP-FULL TRASH CAN,CDOT NEW SIGN,DAMAGE/INVESTIGATE GARB,MISSED BUSINESS GARBAGE,SIDEWALK OBSTRUCT OTHER,SWAT-A-LITTERBUG,ZON VERIFICATION LETTERS
9,BUS STOP-REMOVE TRASH CAN,CDOT NEW STREETLIGHTS,DAMAGE/INVESTIGATE RECY,MISSED GARBAGE,SIGNS (NOT FOR 311 USE),SWS CART DATA LOAD,ZON WATERSHED/SWPE INQ


## Decisions on selection and binning:

- Do we eliminate categories with < N total records?
- Do we combine some categories, for example all 14 "CART ..." categories into one
- Which of the above should we do first?


## Customers with the highest number of requests

In [9]:
# Using address as "customer"
address_ranks = requests['FULL_ADDRESS'].value_counts().head(20)
address_ranks

 PROVIDENCE RD, CHARLOTTE, NC  28211           603
8301 SHARON LAKES RD, CHARLOTTE, NC  28210     563
 E W T HARRIS BV, CHARLOTTE, NC  28213         419
 REA RD, CHARLOTTE, NC  28277                  413
 PARK RD, CHARLOTTE, NC  28210                 412
 S TRYON ST, CHARLOTTE, NC  28262              399
 N TRYON ST, CHARLOTTE, NC  28202              366
 CARMEL RD, CHARLOTTE, NC  28226               350
1120 MARBLE ST, CHARLOTTE, NC  28208           327
3642 HUYTON CT, CHARLOTTE, NC  28215           320
5015 CHERRYCREST LN, CHARLOTTE, NC  28217      310
4624 SHARON CHASE DR, CHARLOTTE, NC  28215     309
5200 AMITY SPRINGS DR, CHARLOTTE, NC  28212    306
1357 SHARON RD WEST, CHARLOTTE, NC  28210      305
6619 YATESWOOD DR, CHARLOTTE, NC  28212        301
1207 KELSTON PL, CHARLOTTE, NC  28212          298
201 S HOSKINS RD, CHARLOTTE, NC  28208         298
2868 HEATHSTEAD PL, CHARLOTTE, NC  28210       298
1701 WEST BV, CHARLOTTE, NC  28208             297
7139 WINDING CEDAR TL, CHARLOTT

In [10]:
# Combining LATTITUDE and LONGITUDE to create unique location IDs accurate to 1.11m to define as "customer"
requests['LATLON'] = requests["LATITUDE"].astype(str) + requests["LONGITUDE"].astype(str)
# Rank customers by frequency of requests
customer_ranks = requests['LATLON'].value_counts().to_frame().reset_index()
customer_ranks.columns = ['LATLON', 'Number of Calls'] # change column names
top_20 = customer_ranks[0:20]
top_20

,LATLON,Number of Calls
0,nannan,72433
1,35.26671306-80.76104322,758
2,35.12039601-80.86962228,563
3,35.23909427-80.92592474,356
4,35.25927032-80.90078488,344
5,35.26371343-80.88607182,320
6,35.26391476-80.73330321,320
7,35.14960744-80.88478039,312
8,35.22968575-80.74488978,309
9,35.19929285-80.76009595,306


## Frequency of Requests Over Time

In [11]:
requests['RECEIVED_DATE'].head()

0    2016/11/07 14:25:00+00
1    2016/11/07 14:28:00+00
2    2016/11/07 14:34:00+00
3    2016/11/07 14:41:00+00
4    2016/11/07 14:44:00+00
Name: RECEIVED_DATE, dtype: object

## Importing Violent Crime Demographics dataset

In [12]:
#importing violent crime demographic data
violent_crime_url='https://bitbucket.org/nthammad-uncc/test_repo/raw/9e2f46eb504bb5a1d863a4dd85e2a531cce4f2fd/data/Violent_Crime_Demographics.csv'
violent_crime_df = read_data_from_URL(violent_crime_url)
violent_crime_df.head()


,OBJECTID,ROW_TYPE,GEOGRAPHY,GEOGRAPHY_ID,CALENDAR_YEAR,CALENDAR_MONTH,OFFENSE_DESCRIPTION,ATTRIBUTE,ATTRIBUTE_VALUE,ATTRIBUTE_COUNT
0,1,CMPD Jurisdiction,CMPD,0,2021,8,Violent Crime,Victim Race/Ethnicity,White,92
1,2,CMPD Jurisdiction,CMPD,0,2021,8,Violent Crime,Victim Race/Ethnicity,Black,395
2,3,CMPD Jurisdiction,CMPD,0,2021,8,Violent Crime,Victim Race/Ethnicity,Hispanic,111
3,4,CMPD Jurisdiction,CMPD,0,2021,8,Violent Crime,Victim Race/Ethnicity,Other/Unknown,8
4,5,CMPD Jurisdiction,CMPD,0,2021,8,Violent Crime,Victim Race/Ethnicity,Asian,13


In [13]:
print("Records:", violent_crime_df.shape[0], "\nFeatures:", violent_crime_df.shape[1])

Records: 143943 
Features: 10


## Importing Census Decennial Race and Yearly Household Income dataset

In [14]:
#importing census economic dataset which contains household income and age data
census_economic_url='https://bitbucket.org/nthammad-uncc/test_repo/raw/93754ccede94c84702bb48cf44939614b4bbc2dc/data/CENSUS_ECONOMIC_2016_2019.csv'
census_df = read_data_from_URL(census_economic_url)
print("Records:", census_df.shape[0], "\nFeatures:", census_df.shape[1])

Records: 932 
Features: 551


In [15]:
census_df.head()

,YEAR,id,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,Percent!!EMPLOYMENT STATUS!!Population 16 years and over,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Margin of Error!!EMPLOYMENT STATUS!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Civilian labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Margin of Error!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Percent Margin of Error!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over,Margin of Error!!EMPLOYMENT STATUS!!Females 16 years and over,Percent!!EMPLOYMENT STATUS!!Females 16 years and over,Percent Margin of Error!!EMPLOYMENT STATUS!!Females 16 years and over,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force,Margin of Error!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force,Percent!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force,Margin of Error!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force,Percent Margin of Error!!EMPLOYMENT STATUS

## Converting feature names from Census data to better format and removing unnecessary features 

In [16]:
# Function to update column names of the census dataset
# Census data set has column names that follow the format:
## 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over'
## 'Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over'
## 'Percent!!EMPLOYMENT STATUS!!Population 16 years and over'
## 'Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over'

# There are also lot of nested details in the column names like: 
## 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces'

# This function updates the Estimate columns in the format:
## EMPLOYMENT STATUS_Population 16 years and over --> removing "Estimate" and including underscore
## Margin of Error_EMPLOYMENT STATUS_Population 16 years and over --> including underscores
## Percent_EMPLOYMENT STATUS_Population 16 years and over --> including underscores
## Percent Margin of Error_EMPLOYMENT STATUS_Population 16 years and over --> including underscores

## [check if it makes sense] EMPLOYMENT STATUS_Population 16 years and over_Armed Forces --> removing unnecessary nesting "Armed Forces"

# The columns starting with "Margin of Error", "Percent" and "Percent Margin of Error" will be dropped later

def rename_census_column(df):
    col_dict={}
    for col_name in df.columns.values.tolist():
        col_split=col_name.split('!!')
        if len(col_split) == 1:
            if col_split[0] == 'id':
                col_dict[col_name]='GEO_ID'
            else:
                col_dict[col_name]=col_name
        elif len(col_split) >=3:
            if col_split[0] == 'Estimate':
                col_dict[col_name]='_'.join(col_split[1:])
            else:
                col_dict[col_name]='_'.join(col_split)
        
    df.rename(col_dict, axis=1, inplace=True)

In [17]:
rename_census_column(census_df)

In [18]:
# This is a list of some unneccessary Estimate columns that need to be dropped
lst_other_col_names=['EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force',
                        'COMMUTING TO WORK_Mean travel time to work (minutes)',
                        'EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate',
                        'EMPLOYMENT STATUS_Females 16 years and over_In labor force',
                        'EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force',
                        'EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed',
                        'EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force',
                        'EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force',
                        'COMMUTING TO WORK_Mean travel time to work (minutes)'
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$15,000 to $24,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$25,000 to $34,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$35,000 to $49,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$50,000 to $74,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$75,000 to $99,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$100,000 to $149,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With earnings',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With earnings_Mean earnings (dollars)',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With Social Security',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With Social Security_Mean Social Security income (dollars)',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With retirement income_Mean retirement income (dollars)',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With Supplemental Security Income',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With Supplemental Security Income_Mean Supplemental Security Income (dollars)',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With cash public assistance income_Mean cash public assistance income (dollars)',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$15,000 to $24,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$25,000 to $34,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$35,000 to $49,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$50,000 to $74,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$75,000 to $99,999',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$100,000 to $149,999', 
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Median earnings for male full-time, year-round workers (dollars)',
                        'INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Median earnings for female full-time, year-round workers (dollars)',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_With health insurance coverage_With private health insurance',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_With health insurance coverage_With public coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population under 18 years',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population under 18 years_No health insurance coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Employed',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Employed_With health insurance coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Employed_With health insurance coverage_With private health insurance',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Employed_With health insurance coverage_With public coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Employed_No health insurance coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed_With health insurance coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed_With health insurance coverage_With private health insurance',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed_With health insurance coverage_With public coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed_No health insurance coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_Not in labor force',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_Not in labor force_With health insurance coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_Not in labor force_With health insurance coverage_With private health insurance',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_Not in labor force_With health insurance coverage_With public coverage',
                        'HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_Not in labor force_No health insurance coverage',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All families_With related children of the householder under 18 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All families_With related children of the householder under 18 years_With related children of the householder under 5 years only',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Married couple families',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Married couple families_With related children of the householder under 18 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Married couple families_With related children of the householder under 18 years_With related children of the householder under 5 years only',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Families with female householder, no husband present',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Families with female householder, no husband present_With related children of the householder under 18 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Families with female householder, no husband present_With related children of the householder under 18 years_With related children of the householder under 5 years only',          
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_Under 18 years_Related children of the householder under 18 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_Under 18 years_Related children of the householder under 18 years_Related children of the householder under 5 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_Under 18 years_Related children of the householder under 18 years_Related children of the householder 5 to 17 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over_18 to 64 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over_65 years and over',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All families',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_Under 18 years',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_People in families',
                        'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Unrelated individuals 15 years and over'
                        ]

In [19]:
# Function to remove unnecessary features in census data
def drop_census_cols(df):
    drop_col_list=[]
    drop_df=df.copy(deep=True)
    lst_first_words=['Margin of Error','Percent','Percent Margin of Error','OCCUPATION','INDUSTRY','CLASS OF WORKER']
    lst_percent_exceptions = ['Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over',
       'Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over_18 to 64 years',
       'Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over_65 years and over',
       'Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_People in families',
       'Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_Unrelated individuals 15 years and over']
    for col_name in drop_df.columns.values.tolist():
        col_split=col_name.split('_')
        if len(col_split)>1 :
            if col_split[0] in lst_first_words and col_name not in lst_percent_exceptions:
                drop_col_list.append(col_name)        
        
        if col_name in lst_other_col_names:
            drop_col_list.append(col_name)
    
    drop_df.drop(drop_col_list, axis=1, inplace=True)
    return drop_df

In [20]:
dropped_census_df=drop_census_cols(census_df)
print("Original Census :::\nRecords:", census_df.shape[0], "\nFeatures:", census_df.shape[1])
print("Dropped Census :::\nRecords:", dropped_census_df.shape[0], "\nFeatures:", dropped_census_df.shape[1])

Original Census :::
Records: 932 
Features: 551
Dropped Census :::
Records: 932 
Features: 51


In [21]:
dropped_census_df.columns.values

array(['YEAR', 'GEO_ID', 'Geographic Area Name',
       'EMPLOYMENT STATUS_Population 16 years and over',
       'EMPLOYMENT STATUS_Population 16 years and over_In labor force',
       'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed',
       'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed',
       'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces',
       'EMPLOYMENT STATUS_Population 16 years and over_Not in labor force',
       'EMPLOYMENT STATUS_Civilian labor force',
       'EMPLOYMENT STATUS_Females 16 years and over',
       'EMPLOYMENT STATUS_Own children of the householder under 6 years',
       'EMPLOYMENT STATUS_Own children of the householder 6 to 17 years',
       'COMMUTING TO WORK_Workers 16 years and over',
       'COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone',
       'COMMUTING TO WORK_Workers 16 years and over_Car, truck

## Transforming census data GEO_ID column

In [22]:
# GEO_ID column of the census data is in the form 1400000US37119000100
# This function transforms the column to only store the census tract id which is 37119000100
# This id will then be used to join the census data with zip codes

def transform_census_geo_id(geoid):
    if 'US' in str(geoid):
        return int(geoid.split('US')[1])
    else:
        return int(geoid)


In [23]:
dropped_census_df['GEO_ID'] = dropped_census_df['GEO_ID'].apply(lambda x: transform_census_geo_id(x))
dropped_census_df['GEO_ID'].head(5)

0    37119000100
1    37119000300
2    37119000400
3    37119000500
4    37119000600
Name: GEO_ID, dtype: int64

## Importing Zip code to Census Tract conversion dataset

In [24]:
#importing census tract and zip code mapping dataset
census_tract_url='https://bitbucket.org/nthammad-uncc/test_repo/raw/4d8285b0ff3d0ae103f0d5c41e8f88d69f160891/data/Zip_to_Census_Track.csv'
census_tract_df = read_data_from_URL(census_tract_url)
census_tract_df.head()

,ZIP,TRACT,USPS_ZIP_PREF_CITY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,27006,37059080200,ADVANCE,0.224640,0.190157,0.483516,0.225580
1,27006,37059080300,ADVANCE,0.577647,0.704698,0.516484,0.583554
2,27006,37059080400,ADVANCE,0.197713,0.105145,0.000000,0.190866
3,27007,37171931102,ARARAT,0.249526,0.344828,0.000000,0.252078
4,27007,37171931101,ARARAT,0.722960,0.551724,0.000000,0.718375


In [25]:
# drop unnecessary column
census_tract_df.drop('USPS_ZIP_PREF_CITY', axis=1, inplace=True)

# rename TRACT to GEO ID and ZIP to ZIP_CODE   
census_tract_df.rename({"TRACT":"GEO_ID", "ZIP": "ZIP_CODE"}, axis=1, inplace=True)

census_tract_df.head()

,ZIP_CODE,GEO_ID,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,27006,37059080200,0.224640,0.190157,0.483516,0.225580
1,27006,37059080300,0.577647,0.704698,0.516484,0.583554
2,27006,37059080400,0.197713,0.105145,0.000000,0.190866
3,27007,37171931102,0.249526,0.344828,0.000000,0.252078
4,27007,37171931101,0.722960,0.551724,0.000000,0.718375


In [26]:
print("Records:", census_tract_df.shape[0], "\nFeatures:", census_tract_df.shape[1])

Records: 5301 
Features: 6


## Merge Census data with zip code

In [27]:
# Each GEO ID can be associated to multiple zip codes and some zip codes can be associated to multiple GEO IDs
# pandas merge ensures each of the GEO IDs are mapped to each zip code and vice-versa
# this might duplicate the data but there is no loss of it
merged_census_df = pd.merge(dropped_census_df, census_tract_df, 
                   on='GEO_ID', 
                   how='left')
merged_census_df.head()

,YEAR,GEO_ID,Geographic Area Name,EMPLOYMENT STATUS_Population 16 years and over,EMPLOYMENT STATUS_Population 16 years and over_In labor force,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,EMPLOYMENT STATUS_Civilian labor force,EMPLOYMENT STATUS_Females 16 years and over,EMPLOYMENT STATUS_Own children of the householder under 6 years,EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,COMMUTING TO WORK_Workers 16 years and over,"COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),COMMUTING TO WORK_Workers 16 years and over_Walked,COMMUTING TO WORK_Workers 16 years and over_Other means,COMMUTING TO WORK_Workers 16 years and over_Worked at home,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households,"INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_Less than $10,000","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$10,000 to $14,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$15,000 to $24,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$150,000 to $199,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$200,000 or more",INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_Median household income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_Mean household income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With retirement income,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With cash public assistance income,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With Food Stamp/SNAP benefits in the past 12 months,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families,"INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_Less than $10,000","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$10,000 to $14,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$150,000 to $199,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$200,000 or more",INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_Median family income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_Mean family income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Per capita income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Nonfamily households,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Nonfamily households_Median nonfamily income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Nonfamily households_Mean nonfamily income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Median earnings for workers (dollars),HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population,HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_With health insurance coverage,HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_No health insurance coverage,Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over,Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over_18 to 64 years,Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over_65 years and over,Percent_PERCENTAGE OF FAMILIES AND PEOPLE WH

In [28]:
print("Merged Census :::\nRecords:", merged_census_df.shape[0], "\nFeatures:", merged_census_df.shape[1])

Merged Census :::
Records: 1748 
Features: 56


## Merge service requests with violent crime

In [29]:
# check for null values in both datasets
violent_crime_df.isnull().sum()

OBJECTID               0
ROW_TYPE               0
GEOGRAPHY              0
GEOGRAPHY_ID           0
CALENDAR_YEAR          0
CALENDAR_MONTH         0
OFFENSE_DESCRIPTION    0
ATTRIBUTE              0
ATTRIBUTE_VALUE        0
ATTRIBUTE_COUNT        0
dtype: int64

In [30]:
requests.isnull().sum()

OBJECTID                          0
DEPARTMENT                        0
REQUEST_NO                        0
FISCAL_YEAR                       0
FISCAL_MONTH                      0
REQUEST_TYPE                      0
RECEIVED_DATE                     0
INTERNAL_FIELD_OBSERVATION        0
TITLE                             0
NEIGHBORHOOD_PROFILE_AREA     72448
BLOCK_NO                      44353
ZIP_CODE                      45083
FULL_ADDRESS                   7168
X_COORD                       72426
Y_COORD                       72426
LATITUDE                      72433
LONGITUDE                     72433
LATLON                            0
dtype: int64

In [31]:
# sort the violent crime dataset by GEOGRAPHY_ID
violent_crime_df.sort_values("GEOGRAPHY_ID").head()

,OBJECTID,ROW_TYPE,GEOGRAPHY,GEOGRAPHY_ID,CALENDAR_YEAR,CALENDAR_MONTH,OFFENSE_DESCRIPTION,ATTRIBUTE,ATTRIBUTE_VALUE,ATTRIBUTE_COUNT
0,1,CMPD Jurisdiction,CMPD,0,2021,8,Violent Crime,Victim Race/Ethnicity,White,92
55658,55659,CMPD Jurisdiction,CMPD,0,2019,3,Violent Crime,Offender Gender,Male,233
55659,55660,CMPD Jurisdiction,CMPD,0,2019,3,Violent Crime,Offender Gender,Female,72
55660,55661,CMPD Jurisdiction,CMPD,0,2019,3,Violent Crime,Offender Race/Ethnici,Other/Unknown,1
55661,55662,CMPD Jurisdiction,CMPD,0,2019,3,Violent Crime,Offender Race/Ethnici,White,34


In [32]:
violent_crime_df['ATTRIBUTE'].value_counts()

Victim Age Group         32981
Victim Race/Ethnicity    22462
Victim Gender            21611
Offender Age Group       19669
Domestic Violence        18652
Offender Gender          14523
Offender Race/Ethnici    14045
Name: ATTRIBUTE, dtype: int64

In [33]:
# sort the requests dataset by NEIGHBORHOOD_PROFILE_AREA 
requests.sort_values("NEIGHBORHOOD_PROFILE_AREA").head()

,OBJECTID,DEPARTMENT,REQUEST_NO,FISCAL_YEAR,FISCAL_MONTH,REQUEST_TYPE,RECEIVED_DATE,INTERNAL_FIELD_OBSERVATION,TITLE,NEIGHBORHOOD_PROFILE_AREA,BLOCK_NO,ZIP_CODE,FULL_ADDRESS,X_COORD,Y_COORD,LATITUDE,LONGITUDE,LATLON
278398,278399,Solid Waste Services,7049072,2019,01-Jul,MISSED BULKY ITEM,2018/07/02 13:01:00+00,N,MISSED BULKY ITEM,2.0,1032.0,28205.0,"1032 COLISEUM DR, CHARLOTTE, NC 28205",1464941.0,534490.0,35.205346,-80.791311,35.20534595-80.79131058
1392427,1392602,Solid Waste Services,8071411,2021,05-Nov,NON_RECYCLABLE ITEMS,2020/11/27 17:21:00+00,N,Non-recyclable Items,2.0,3133.0,28205.0,"3133 WESTMORELAND AV, CHARLOTTE, NC 28205",1464109.0,535590.0,35.208327,-80.794162,35.20832662-80.79416196
362554,362555,Solid Waste Services,6565893,2017,10-Apr,NON_RECYCLABLE ITEMS,2017/04/28 19:54:00+00,N,NON_RECYCLABLE ITEMS,2.0,3132.0,28205.0,"3132 COMMONWEALTH AV, CHARLOTTE, NC 28205",1464209.0,535801.0,35.208911,-80.793840,35.20891125-80.79384
362538,362539,Solid Waste Services,6565636,2017,10-Apr,NON_RECYCLABLE ITEMS,2017/04/28 17:50:00+00,N,NON_RECYCLABLE ITEMS,2.0,3218.0,28205.0,"3218 BARNHILL DR, CHARLOTTE, NC 28205",1464860.0,535917.0,35.209262,-80.791668,35.20926223-80.79166792
522358,522359,Solid Waste Services,7756220,2020,08-Feb,NON_RECYCLABLE ITEMS,2020/02/20 23:31:12+00,N,Non-recyclable Items,2.0,3626.0,28205.0,"3626 COMMONWEALTH AV, CHARLOTTE, NC 28205",1465812.0,534200.0,35.204592,-80.788378,35.20459241-80.78837773


In [34]:
# creating a new column in the requests dataset to get the month of the request received date
requests['RECEIVED_MONTH']= pd.to_datetime(requests['RECEIVED_DATE']).dt.month
requests['RECEIVED_YEAR']= pd.to_datetime(requests['RECEIVED_DATE']).dt.year

In [35]:
requests.columns.values

array(['OBJECTID', 'DEPARTMENT', 'REQUEST_NO', 'FISCAL_YEAR',
       'FISCAL_MONTH', 'REQUEST_TYPE', 'RECEIVED_DATE',
       'INTERNAL_FIELD_OBSERVATION', 'TITLE', 'NEIGHBORHOOD_PROFILE_AREA',
       'BLOCK_NO', 'ZIP_CODE', 'FULL_ADDRESS', 'X_COORD', 'Y_COORD',
       'LATITUDE', 'LONGITUDE', 'LATLON', 'RECEIVED_MONTH',
       'RECEIVED_YEAR'], dtype=object)

In [36]:
# view the 'NEIGHBORHOOD_PROFILE_AREA', 'FISCAL_YEAR', RECEIVED_MONTH' columns in the requests dataframe
requests[['NEIGHBORHOOD_PROFILE_AREA','RECEIVED_YEAR','RECEIVED_MONTH']].head(10)

,NEIGHBORHOOD_PROFILE_AREA,RECEIVED_YEAR,RECEIVED_MONTH
0,215.0,2016,11
1,265.0,2016,11
2,103.0,2016,11
3,158.0,2016,11
4,NaN,2016,11
5,88.0,2016,11
6,97.0,2016,11
7,76.0,2016,11
8,372.0,2017,8
9,251.0,2017,8


In [37]:
# view the 'GEOGRAPHY_ID','CALENDAR_YEAR','CALENDAR_MONTH' columns in the violent crime dataframe
violent_crime_df[['GEOGRAPHY_ID','CALENDAR_YEAR','CALENDAR_MONTH']].head(10)

,GEOGRAPHY_ID,CALENDAR_YEAR,CALENDAR_MONTH
0,0,2021,8
1,0,2021,8
2,0,2021,8
3,0,2021,8
4,0,2021,8
5,0,2021,8
6,0,2021,8
7,0,2021,8
8,0,2021,8
9,0,2021,8


In [38]:
#check if neighborhood profile area and geography id have any common values
len(list(set(requests.NEIGHBORHOOD_PROFILE_AREA) & set(violent_crime_df.GEOGRAPHY_ID) ))

411

In [39]:
# since there are 411 common values, we can use this column as a merge column 
# while trying to join the requests data frame with violent crime dataframe

# but we need to also check the calendar year and calendar month

# The rest of the columns in the violent crime dataset are not important
# So we can skip these columns 'OBJECTID', 'ROW_TYPE', 'GEOGRAPHY', 'OFFENSE_DESCRIPTION','ATTRIBUTE', 'ATTRIBUTE_VALUE'

# additionally, we need to aggregate the ATTRIBUTE COUNT column in the violent crime dataframe, 
# before we merge the two dataframes

violent_crime_sum_df = violent_crime_df.groupby(['CALENDAR_YEAR','CALENDAR_MONTH','GEOGRAPHY_ID']).agg({'ATTRIBUTE_COUNT': 'sum'}).reset_index()
violent_crime_sum_df.head(10)

,CALENDAR_YEAR,CALENDAR_MONTH,GEOGRAPHY_ID,ATTRIBUTE_COUNT
0,2015,1,0,3131
1,2015,1,2,4
2,2015,1,3,4
3,2015,1,5,4
4,2015,1,6,35
5,2015,1,9,14
6,2015,1,10,22
7,2015,1,12,18
8,2015,1,16,4
9,2015,1,17,4


In [40]:
# create a 'COL_MERGE' column with the format CALENDAR YEAR_CALENDAR MONTH_GEOGRAPHY ID
violent_crime_sum_df['COL_MERGE']=violent_crime_sum_df['CALENDAR_YEAR'].astype(str)+'_'+violent_crime_sum_df['CALENDAR_MONTH'].astype(str)+'_'+violent_crime_sum_df['GEOGRAPHY_ID'].astype(int).astype(str)
violent_crime_sum_df.head(10)

,CALENDAR_YEAR,CALENDAR_MONTH,GEOGRAPHY_ID,ATTRIBUTE_COUNT,COL_MERGE
0,2015,1,0,3131,2015_1_0
1,2015,1,2,4,2015_1_2
2,2015,1,3,4,2015_1_3
3,2015,1,5,4,2015_1_5
4,2015,1,6,35,2015_1_6
5,2015,1,9,14,2015_1_9
6,2015,1,10,22,2015_1_10
7,2015,1,12,18,2015_1_12
8,2015,1,16,4,2015_1_16
9,2015,1,17,4,2015_1_17


In [41]:
# create a COL_MERGE column in the requests dataset too

# check the count of values in the requests dataset that have a valid NEIGHBORHOOD PROFILE AREA ID (not equal to 0 and Nan)
requests[requests.NEIGHBORHOOD_PROFILE_AREA > 0].shape[0]

1559062

In [42]:
requests['NEIGHBORHOOD_PROFILE_AREA'].isnull().sum()

72448

In [43]:
# The 72448 values are NaNs so we convert those values into -1  
# They will have a COL_MERGE value like 2017_11_-1 which will not match with any value in voilent crime
requests['NEIGHBORHOOD_PROFILE_AREA'] = requests['NEIGHBORHOOD_PROFILE_AREA'].fillna(-1)

# create a 'COL_MERGE' column similar to violent crime with the format 
# FISCAL YEAR_RECEIVED MONTH_NEIGHBORHOOD PROFILE AREA

requests['COL_MERGE']=requests['RECEIVED_YEAR'].astype(str)+'_'+requests['RECEIVED_MONTH'].astype(str)+'_'+requests['NEIGHBORHOOD_PROFILE_AREA'].astype(int).astype(str)
requests.head()

,OBJECTID,DEPARTMENT,REQUEST_NO,FISCAL_YEAR,FISCAL_MONTH,REQUEST_TYPE,RECEIVED_DATE,INTERNAL_FIELD_OBSERVATION,TITLE,NEIGHBORHOOD_PROFILE_AREA,BLOCK_NO,ZIP_CODE,FULL_ADDRESS,X_COORD,Y_COORD,LATITUDE,LONGITUDE,LATLON,RECEIVED_MONTH,RECEIVED_YEAR,COL_MERGE
0,1,Solid Waste Services,6402056,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:25:00+00,N,NON_RECYCLABLE ITEMS,215.0,3601.0,28226.0,"3601 KNAPDALE LN, CHARLOTTE, NC 28226",1458251.0,508075.0,35.132444,-80.812089,35.132444-80.81208881,11,2016,2016_11_215
1,2,Housing and NBHD Serv,6402064,2017,05-Nov,UTILITY VERIFICATION LTR,2016/11/07 14:28:00+00,N,UTILITY VERIFICATION LTR,265.0,4124.0,28269.0,"4124 SILVERMERE WY, CHARLOTTE, NC 28269",1468687.0,587226.0,35.350408,-80.781941,35.35040753-80.78194101,11,2016,2016_11_265
2,3,Solid Waste Services,6402082,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:34:00+00,N,Non-recyclable Items,103.0,523.0,28213.0,"523 GRAHAM MEADOW DR, CHARLOTTE, NC 28213",1471481.0,562513.0,35.282654,-80.771094,35.28265369-80.77109378,11,2016,2016_11_103
3,4,Solid Waste Services,6402103,2017,05-Nov,MISSED RECYCLING,2016/11/07 14:41:00+00,N,MISSED RECYCLING,158.0,1801.0,28216.0,"1801 CRANDON DR, CHARLOTTE, NC 28216",1438992.0,568604.0,35.297745,-80.880327,35.29774507-80.88032735,11,2016,2016_11_158
4,5,Transit,6402111,2017,05-Nov,CITIZEN FEEDBACK,2016/11/07 14:44:00+00,N,CITIZEN FEEDBACK,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nannan,11,2016,2016_11_-1


In [44]:
#merge the service requests dataframe with violent crime sum dataframe
merged_req_vc_df = pd.merge(requests, violent_crime_sum_df, 
                   on='COL_MERGE', 
                   how='left')
print("Records:", merged_req_vc_df.shape[0], "\nFeatures:", merged_req_vc_df.shape[1])

Records: 1631510 
Features: 25


In [45]:
merged_req_vc_df.head(10)

,OBJECTID,DEPARTMENT,REQUEST_NO,FISCAL_YEAR,FISCAL_MONTH,REQUEST_TYPE,RECEIVED_DATE,INTERNAL_FIELD_OBSERVATION,TITLE,NEIGHBORHOOD_PROFILE_AREA,BLOCK_NO,ZIP_CODE,FULL_ADDRESS,X_COORD,Y_COORD,LATITUDE,LONGITUDE,LATLON,RECEIVED_MONTH,RECEIVED_YEAR,COL_MERGE,CALENDAR_YEAR,CALENDAR_MONTH,GEOGRAPHY_ID,ATTRIBUTE_COUNT
0,1,Solid Waste Services,6402056,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:25:00+00,N,NON_RECYCLABLE ITEMS,215.0,3601.0,28226.0,"3601 KNAPDALE LN, CHARLOTTE, NC 28226",1458251.0,508075.0,35.132444,-80.812089,35.132444-80.81208881,11,2016,2016_11_215,NaN,NaN,NaN,NaN
1,2,Housing and NBHD Serv,6402064,2017,05-Nov,UTILITY VERIFICATION LTR,2016/11/07 14:28:00+00,N,UTILITY VERIFICATION LTR,265.0,4124.0,28269.0,"4124 SILVERMERE WY, CHARLOTTE, NC 28269",1468687.0,587226.0,35.350408,-80.781941,35.35040753-80.78194101,11,2016,2016_11_265,NaN,NaN,NaN,NaN
2,3,Solid Waste Services,6402082,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:34:00+00,N,Non-recyclable Items,103.0,523.0,28213.0,"523 GRAHAM MEADOW DR, CHARLOTTE, NC 28213",1471481.0,562513.0,35.282654,-80.771094,35.28265369-80.77109378,11,2016,2016_11_103,2016.0,11.0,103.0,31.0
3,4,Solid Waste Services,6402103,2017,05-Nov,MISSED RECYCLING,2016/11/07 14:41:00+00,N,MISSED RECYCLING,158.0,1801.0,28216.0,"1801 CRANDON DR, CHARLOTTE, NC 28216",1438992.0,568604.0,35.297745,-80.880327,35.29774507-80.88032735,11,2016,2016_11_158,2016.0,11.0,158.0,11.0
4,5,Transit,6402111,2017,05-Nov,CITIZEN FEEDBACK,2016/11/07 14:44:00+00,N,CITIZEN FEEDBACK,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nannan,11,2016,2016_11_-1,NaN,NaN,NaN,NaN
5,6,Finance/City,6402112,2017,05-Nov,CWP REQUEST,2016/11/07 14:45:00+00,N,CWP REQUEST-NORMAL,88.0,309.0,28208.0,"309 JONES ST APT 1, CHARLOTTE, NC 28208",1439454.0,552715.0,35.254120,-80.877772,35.25411956-80.877772,11,2016,2016_11_88,2016.0,11.0,88.0,4.0
6,7,Solid Waste Services,6402126,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:49:33+00,N,Non-recyclable Items,97.0,11813.0,28273.0,"11813 BROADWATER LN, CHARLOTTE, NC 28273",1410991.0,500986.0,35.110496,-80.969658,35.11049568-80.96965789,11,2016,2016_11_97,NaN,NaN,NaN,NaN
7,8,Solid Waste Services,6402140,2017,05-Nov,NON_RECYCLABLE ITEMS,2016/11/07 14:53:00+00,N,NON_RECYCLABLE ITEMS,76.0,13801.0,28278.0,"13801 S TRYON ST, CHARLOTTE, NC 28278",1401313.0,495906.0,35.096008,-81.001670,35.09600819-81.00167012,11,2016,2016_11_76,NaN,NaN,NaN,NaN
8,9,Solid Waste Services,6714903,2018,02-Aug,NON_RECYCLABLE ITEMS,2017/08/30 17:55:51+00,N,Non-recyclable Items,372.0,4513.0,28269.0,"4513 ROCK STREAM DR, CHARLOTTE, NC 28269",1470306.0,596868.0,35.376975,-80.777091,35.37697539-80.77709113,8,2017,2017_8_372,NaN,NaN,NaN,NaN
9,10,Solid Waste Services,6714919,2018,02-Aug,NON_RECYCLABLE ITEMS,2017/08/30 18:00:00+00,N,NON_RECYCLABLE ITEMS,251.0,5131.0,28269.0,"5131 EAGLE CREEK DR, CHARLOTTE, NC 28269",1465887.0,591565.0,35.362190,-80.791593,35.36219004-80.79159283,8,2017,2017_8_251,NaN,NaN,NaN,NaN


## Merge Service Request & Violent crime with Census data

In [46]:
merged_census_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1748 entries, 0 to 1747
Data columns (total 56 columns):
 #   Column                                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                                  --------------  -----  
 0   YEAR                                                                                                                                                    1748 non-null   int64  
 1   GEO_ID                                                                                                                                                  1748 non-null   int64  
 2   Geographic Area Name                                                                                                                                    1748 non-nul

In [47]:
merged_req_vc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1631510 entries, 0 to 1631509
Data columns (total 25 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   OBJECTID                    1631510 non-null  int64  
 1   DEPARTMENT                  1631510 non-null  object 
 2   REQUEST_NO                  1631510 non-null  int64  
 3   FISCAL_YEAR                 1631510 non-null  int64  
 4   FISCAL_MONTH                1631510 non-null  object 
 5   REQUEST_TYPE                1631510 non-null  object 
 6   RECEIVED_DATE               1631510 non-null  object 
 7   INTERNAL_FIELD_OBSERVATION  1631510 non-null  object 
 8   TITLE                       1631510 non-null  object 
 9   NEIGHBORHOOD_PROFILE_AREA   1631510 non-null  float64
 10  BLOCK_NO                    1587157 non-null  float64
 11  ZIP_CODE                    1586427 non-null  float64
 12  FULL_ADDRESS                1624342 non-null  object 
 1

In [48]:
merged_req_vc_df['RECEIVED_YEAR'].value_counts()

2019    330447
2020    325535
2018    318578
2017    299860
2021    209965
2016    147125
Name: RECEIVED_YEAR, dtype: int64

In [49]:
merged_census_df['YEAR'].value_counts()

2019    437
2018    437
2017    437
2016    437
Name: YEAR, dtype: int64

In [50]:
# both merged_census_df and merged_req_vc_df have ZIP_CODE column and YEAR column 
# so we can use them to join these two dataframes

# census data is available until 2019. 
# The data is has some issues during the pandemic 
# so requests received in 2020 and 2021 will be mapped to the census year 2019

# creating an empty column ZIP_YEAR
merged_req_vc_df['ZIP_YEAR']=np.nan
merged_census_df['ZIP_YEAR']=np.nan

In [51]:
merged_req_vc_df['ZIP_YEAR']=merged_req_vc_df[['RECEIVED_YEAR','ZIP_CODE']].apply(lambda x: '%s_%s' % (x['RECEIVED_YEAR'].astype(int) if x.RECEIVED_YEAR<=2019 else 2019,0 if pd.isnull(x['ZIP_CODE']) else x['ZIP_CODE'].astype(int)) , axis=1)
merged_req_vc_df['ZIP_YEAR'].head()

0    2016_28226
1    2016_28269
2    2016_28213
3    2016_28216
4        2016_0
Name: ZIP_YEAR, dtype: object

In [52]:
merged_census_df['ZIP_YEAR']=merged_census_df[['YEAR','ZIP_CODE']].apply(lambda x: '%s_%s' % (x['YEAR'].astype(int),x['ZIP_CODE'].astype(int)), axis=1)


In [53]:
# merge both the dataframes on ZIP_YEAR column
merged_requests_df = pd.merge(merged_req_vc_df, merged_census_df, 
                   on='ZIP_YEAR', 
                   how='left')
print("Records:", merged_requests_df.shape[0], "\nFeatures:", merged_requests_df.shape[1])

Records: 25143751 
Features: 82


In [54]:
merged_requests_df.head()

,OBJECTID,DEPARTMENT,REQUEST_NO,FISCAL_YEAR,FISCAL_MONTH,REQUEST_TYPE,RECEIVED_DATE,INTERNAL_FIELD_OBSERVATION,TITLE,NEIGHBORHOOD_PROFILE_AREA,BLOCK_NO,ZIP_CODE_x,FULL_ADDRESS,X_COORD,Y_COORD,LATITUDE,LONGITUDE,LATLON,RECEIVED_MONTH,RECEIVED_YEAR,COL_MERGE,CALENDAR_YEAR,CALENDAR_MONTH,GEOGRAPHY_ID,ATTRIBUTE_COUNT,ZIP_YEAR,YEAR,GEO_ID,Geographic Area Name,EMPLOYMENT STATUS_Population 16 years and over,EMPLOYMENT STATUS_Population 16 years and over_In labor force,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,EMPLOYMENT STATUS_Civilian labor force,EMPLOYMENT STATUS_Females 16 years and over,EMPLOYMENT STATUS_Own children of the householder under 6 years,EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,COMMUTING TO WORK_Workers 16 years and over,"COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),COMMUTING TO WORK_Workers 16 years and over_Walked,COMMUTING TO WORK_Workers 16 years and over_Other means,COMMUTING TO WORK_Workers 16 years and over_Worked at home,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households,"INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_Less than $10,000","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$10,000 to $14,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$15,000 to $24,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$150,000 to $199,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_$200,000 or more",INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_Median household income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Total households_Mean household income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With retirement income,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With cash public assistance income,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_With Food Stamp/SNAP benefits in the past 12 months,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families,"INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_Less than $10,000","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$10,000 to $14,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$150,000 to $199,999","INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_$200,000 or more",INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_Median family income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Families_Mean family income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Per capita income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Nonfamily households,INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Nonfamily households_Median nonfamily income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Nonfamily households_Mean nonfamily income (dollars),INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)_Median earnings for workers (dollars),HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population,HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_With health insurance coverage,HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_No health insurance coverage,Percent_PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL_All people_18 years and over,Percent_PERCENTAGE O

<h2> Data Preprocessing </h2>

<h2> Data Understanding and Exploration </h2>

<h2> Data Preparation for Modeling </h2>

<h2> Modeling </h2>

<h2> Evaluation </h2>

<h2> Results </h2>